In [1]:
include("helpers.jl")

using Revise
using SparseArrays
using LinearAlgebra

using JuMP
import SparseArrays
using LinearAlgebra
import Convex
using SCIP
using HiGHS
using Pipe: @pipe
using DataFrames
import CSV

In [2]:


NUM_VERTS = 6
# NUM_VERTS = 9

filepath = "QAPN6LP_5.lp"

"QAPN6LP_5.lp"

In [ ]:
# read the model from the LP file and convert it into normal form A x == b ; x >= 0
# slack variables are added if necessary, but they're only needed for the new y-var extension

m = JuMP.read_from_file(filepath)
A, b, var_names = really_normal_form(m)

In [ ]:
var_idx_tuples = var_name_to_tuple.(var_names)
var_idx_tuple_lookup = Dict(var_idx_tuples[i] => i for i in 1:length(var_names))

In [ ]:
# generate the "standard" extreme points to the polytope, one for each valid TSP path. This code uses a sparse method
# for efficiency but in principle this can be achieved by iterating through all m! paths, generating an objective function for each 
# path, solving the LP with it and recording the solution.
sparse_points = SparseArrays.sparse(compute_verts(NUM_VERTS - 1)..., factorial(NUM_VERTS - 1), length(var_idx_tuples))

# Verify that these points all satisfy the constraints
@assert norm(A * sparse_points' .- b) < 1e-9



In [ ]:
# for visualization, display the standard points
df = DataFrame(Tables.table(sparse_points; header=var_names))
CSV.write("output/standard_points.csv", df)
df

In [ ]:
# this is the function that finds candidate nonintegral extreme points.
# as an input it takes a list of standard extreme point indices, whose weights we'll set to -1 and then try to cover 
function find_complements(neg_vertex_idxes :: Vector{Int}; silent=true)
    @assert issorted(neg_vertex_idxes)    
    neg_components = sparse_points[neg_vertex_idxes, :]'
    neg_sum = SparseArrays.sparse(sum(neg_components, dims = 2))[:, 1]

    # Let N be the number of standard EPs that haven't been fixed to -1, and
    # p_i be those standard standard EPs. 
    # In principle, we want to solve the following optimization problem:
    #
    # Integer variables α_i , i ∈ [1, m! - length(neg_vertex_idxes)]
    #
    # minimize ∑ α_i
    # s.t. 
    #     α >= 0
    #     ∑ α_i p_i >= neg_sum
    #
    #
    # But for efficiency reasons, since the standard EPs are very sparse,
    # instead of using full points we only look at the columns in which neg_sum is nonempty.

    nonzero_column_indices = findall(x -> x > 0, neg_sum)

    neg_index_mask = zeros(Bool, factorial(NUM_VERTS-1))
    neg_index_mask[neg_vertex_idxes] .= true

    reduced_candidate_points = sparse_points[.~(neg_index_mask), :][:, nonzero_column_indices]

    # here we define the actual optimization problem
    assignments = Convex.Variable(factorial(NUM_VERTS-1)-length(neg_vertex_idxes), Convex.IntVar)
    constraints = [
        assignments >= 0,
        reduced_candidate_points' * assignments >= neg_sum[nonzero_column_indices]
    ]
    problem = Convex.minimize(sum(assignments), constraints)
    Convex.solve!(problem, SCIP.Optimizer; silent_solver=silent)
    write_convex_jl_model_to_file(problem, "output/set_cover.lp")

    # now, some bookkeeping to form the α vector with both positive and negative coefficients
    assignmet_value = Convex.evaluate(assignments)
    for (neg_idx, offset) in zip(neg_vertex_idxes, 0:length(neg_vertex_idxes)-1)
        insert!(assignmet_value, neg_idx, -1)
    end

    assignment_value_norm = round.(assignmet_value)
    assignment_value_norm /= sum(assignment_value_norm)

    write("output/weights_of_new_point.txt", repr(sparse(assignment_value_norm)))
    
    res_point = sparse_points' * assignment_value_norm

    @assert norm(A * res_point .- b) < 1e-9 "Not feasive"
    @assert all(res_point .>= -1e-9) "Not nonnegative"
    @assert rank(A[:, res_point .> 1e-9]) == sum(res_point .> 1e-9; init=0) "Not a basis solution"
    
    sparse(assignment_value_norm), sparse(res_point)
end

In [ ]:
# Now, we can choose which standard EPs to fix to -1. Choosing just one EP, e.g. point 1, produces a non-integral extreme point.
alpha_values, nonint_ep = find_complements([1]; silent=true)
@assert sum(alpha_values) ≈ 1.0
nonint_ep


In [ ]:
# In fact, sometimes fixing more than one standard EP to -1 gives a new non-integral EP:
_, res_point_extra = find_complements([22, 44]; silent=true)
res_point_extra

In [ ]:
# But other times it doesn't produce an extreme point: either there's no way to cover the fixed-negative vectors,
# or we still get a feasible point but it's not a vertex of the polytope.
find_complements([1, 2]; silent=true)



In [ ]:
# Anyway, to double-check our work, we construct an objective function for the original LP for which the optimal point is attained at our new non-integral EP.
# In this case, we can just set the objective c = -res_point . (In some cases this doesn't work and we need to solve an extra LP, exposition for that case is TODO)

x = Convex.Variable(length(nonint_ep))
problem = Convex.minimize(
    -nonint_ep' * x,
    [
        A * x == b,
        x >= 0
    ]    
)
Convex.solve!(problem, SCIP.Optimizer)
solution = round.(sparse(Convex.evaluate(x)); digits=9)

# And indeed, we see that the optimal value is attained at a non-integral point


In [ ]:
# Finally, we can write an objective function with the old variable names:

obj_string = let
    format_num_name_to_string(name, num) = begin
        sign = num < 0 ? "" : "+"
        "$sign $num $name"
    end 
    parts = @pipe zip(var_names, -nonint_ep) |> collect |> filter(x -> x[2] != 0) |> map(x -> format_num_name_to_string(x...), _)
    reduce(*, parts)
end
write("output/obj_function_for_nonintegral_sln.txt", "Minimize\n  obj: $obj_string")